In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.magic import register_cell_magic

from y2klc3tools.sqlvm import SqlVM

In [ ]:
vm = SqlVM()


@register_cell_magic
def sql(_, cell):
    vm.run_and_trace(cell)

In [ ]:
vm.load_binary_from_file("../obj/vm/hello2.obj")
vm.reset()
vm.continue_()

In [ ]:
%%sql
-- drive the clock signal
UPDATE signal SET clk = 1;
UPDATE signal SET clk = 0;
UPDATE signal SET clk = 1;
UPDATE signal SET clk = 0;
UPDATE signal SET clk = 0;
UPDATE signal SET clk = 0;
UPDATE signal SET clk = 1;
UPDATE signal SET clk = 1;
UPDATE signal SET clk = 0;
UPDATE signal SET clk = 1;